In [1]:
import blpapi
from blpapi import SessionOptions, Session
import datetime
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from xbbg import blp
from pandas.tseries.offsets import DateOffset
from scipy import stats
from sklearn.linear_model import LinearRegression
import Haver

In [2]:
import warnings

# Suppress FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
class Index_to_stationary:
    # the point of this class is to get data from bloomberg and if that data is not stationary convert it to the 6 month over 6 month change. From there we convert them to z scores. 
    def __init__(self, bbg_ticker):
        #this section allows me to maintian the features throughout the class and instantly lets me pull these features into other classes later wihtout ever defining it. 
        self.index_value = blp.bdh(tickers=bbg_ticker, flds=['PX_LAST'],start_date='1970-01-01',Per='D', Fill='P', Days='A')
        self.ticker = bbg_ticker

    def invert_index_value(self):
        #this sections isto invert an inde to align directional to toehr indicies. A decline in unemployment should be positive for the employment inde and then needs to be inverted
        self.index_value = self.index_value*-1
    
    def mean_revert_trans(self):
        # This mean revesion is the idea that we need our data to be stationary, so we make the data the 6 month change from data 6 months prior. SImilar to year over year, just shorter. 
        self.index_value.index = pd.to_datetime(self.index_value.index) #convert data index to be time based
        df_daily = self.index_value[self.ticker].copy()
        df_daily_copy =df_daily.copy()
        df_daily_copy=df_daily_copy.shift(freq= DateOffset(months=6)) #move data back 6 months
        df_daily_copy=df_daily_copy.rename(columns={'PX_LAST': '6_months_ago'})
        df_daily = pd.merge(df_daily,df_daily_copy, left_index=True, right_index=True, how = 'outer') # this mergers allows me to have a data frame with today's data and 6 months ago to carry out the change operation
        date_before = pd.Timestamp('today')
        df_daily1 = df_daily[df_daily.index <= date_before]
        df_daily1 = df_daily1[~df_daily1.index.duplicated(keep='last')] #sometimes the procedure creates duplicates
        df_daily1 = df_daily1.fillna(method='ffill') #this fills in any weekend gaps
        # Calculate the change over the last 6 months
        df_daily1['6_month_change'] = ((df_daily1['PX_LAST'] - df_daily1['6_months_ago'])/df_daily1['6_months_ago'])

        return df_daily1

    def z_scores(self):
        #here we simply calcualte the z score
        df_daily = self.mean_revert_trans().copy()
        df_daily['Zscore'] = (df_daily['6_month_change'] - df_daily['6_month_change'].mean()) / df_daily['6_month_change'].std()
        df_daily = df_daily[~df_daily.index.duplicated(keep='first')]
        zscore_df = df_daily.drop(['PX_LAST',"6_month_change",'6_months_ago'], axis=1)
        #df_daily['6_month_change'].apply(stats.zscore)
        # Manually calculating Z-scores: z_scores_manual = (df - df.mean()) / df.std()
        return zscore_df
        

In [4]:
class Datapoints_to_stationary:
    #this class is an expansion on the prior class, it allows for the conversation to stationary and then to z scores for bloomberg data that is not just the last price of an index.
    #Used mainly for profits and market bubble index
    def __init__(self, bbg_ticker,field):
        #this section allows me to maintian the features throughout the class and instantly lets me pull these features into other classes later wihtout ever defining it. 
        self.index_value = blp.bdh(tickers=bbg_ticker, flds=field,start_date='1970-01-01',Per='D', Fill='P', Days='A')
        self.ticker = bbg_ticker
        self.field = field

    def mean_revert_trans(self):
        # Calculate the value from 6 months ago (daily data)
        self.index_value.index = pd.to_datetime(self.index_value.index)
        df_daily = self.index_value[self.ticker].copy()
        df_daily_copy =df_daily.copy()
        df_daily_copy=df_daily_copy.shift(freq= DateOffset(months=6))
        df_daily_copy=df_daily_copy.rename(columns={self.field: '6_months_ago'})
        df_daily = pd.merge(df_daily,df_daily_copy, left_index=True, right_index=True, how = 'outer')
        date_before = pd.Timestamp('today')
        df_daily1 = df_daily[df_daily.index <= date_before]
        df_daily1 = df_daily1[~df_daily1.index.duplicated(keep='first')]
        df_daily1 = df_daily1.fillna(method='ffill')
        # Calculate the change over the last 6 months
        df_daily1['6_month_change'] = ((df_daily1[self.field] - df_daily1['6_months_ago'])/df_daily1['6_months_ago'])

        return df_daily1

    def z_scores(self):
        df_daily = self.mean_revert_trans().copy()
        df_daily['Zscore'] = (df_daily['6_month_change'] - df_daily['6_month_change'].mean()) / df_daily['6_month_change'].std()
        df_daily = df_daily[~df_daily.index.duplicated(keep='first')]
        zscore_df = df_daily.drop([self.field,"6_month_change",'6_months_ago'], axis=1)
        #df_daily['6_month_change'].apply(stats.zscore)
        # Manually calculating Z-scores: z_scores_manual = (df - df.mean()) / df.std()
        return zscore_df
        

In [5]:
class Index_that_are_stationary:
    #this is the same idea as indexes before, but exlcduing the conversation for stationary if the index is already stationary like a PMI
    def __init__(self, bbg_ticker):
        #this section allows me to maintian the features throughout the class and instantly lets me pull these features into other classes later wihtout ever defining it. 
        self.index_value = blp.bdh(tickers=bbg_ticker, flds=['PX_LAST'],start_date='1970-01-01',Per='D', Fill='P', Days='A')
        self.ticker = bbg_ticker

    def invert_index_value(self):
        self.index_value = self.index_value*-1
    
    def z_scores(self):
        self.index_value.index = pd.to_datetime(self.index_value.index)
        df_daily = self.index_value[self.ticker].copy()
        df_daily['Zscore'] = (df_daily['PX_LAST'] - df_daily['PX_LAST'].mean()) / df_daily['PX_LAST'].std()
        df_daily = df_daily[~df_daily.index.duplicated(keep='first')]
        zscore_df = df_daily.drop(['PX_LAST'], axis=1)
        #df_daily['6_month_change'].apply(stats.zscore)
        # Manually calculating Z-scores: z_scores_manual = (df - df.mean()) / df.std()
        return zscore_df
        
        

In [6]:
class Datapoints_that_are_stationary:
    def __init__(self, bbg_ticker,field):
        #this section allows me to maintian the features throughout the class and instantly lets me pull these features into other classes later wihtout ever defining it. 
        self.index_value = blp.bdh(tickers=bbg_ticker, flds=field,start_date='1970-01-01',Per='D', Fill='P', Days='A')
        self.ticker = bbg_ticker
        self.field = field

    def invert_index_value(self):
        self.index_value = self.index_value*-1
        
    def z_scores(self):
        self.index_value.index = pd.to_datetime(self.index_value.index)
        df_daily = self.index_value[self.ticker].copy()
        df_daily['Zscore'] = (df_daily[self.field] - df_daily[self.field].mean()) / df_daily[self.field].std()
        df_daily = df_daily[~df_daily.index.duplicated(keep='first')]
        zscore_df = df_daily.drop([self.field], axis=1)
        #df_daily['6_month_change'].apply(stats.zscore)
        # Manually calculating Z-scores: z_scores_manual = (df - df.mean()) / df.std()
        return zscore_df
        
        

In [7]:
class HF_Index:
    #this class is how we make our headline high frequency indciators, by combining the sub indicies. 
    def __init__(self):
        self.datapoints = [] 
    
    def add_data(self, datapoint):
        #this adds the indicators to the class
        self.datapoints.append(datapoint)
    
    def combine_zscores(self):
       # this is where we make a dataframe or table of all the z scores from the indicators we've added. 
        for data in self.datapoints:
            zscore_dfsource = data.z_scores() #this loop is just to make the dataframe, starting by getting each sub indicies z scores 
            zscore_df = zscore_dfsource.copy()
            if isinstance(data, Datapoints_that_are_stationary) or isinstance(data, Datapoints_to_stationary): #this if statement checks what class it is in, so I can rename columns and not have duplicates
                zscore_df.rename(columns={'Zscore': (data.ticker + " " + data.field)}, inplace=True)
            else:
                zscore_df.rename(columns={'Zscore': data.ticker}, inplace=True)
                
            if data == self.datapoints[0]:
                combined_df = zscore_df.copy()
            
            else:
                combined_df = pd.merge(combined_df,  zscore_df, left_index=True, right_index=True,how='outer')
    
        
        return combined_df 
    
    def highfrequency_index(self, HF_Index_Name_string): #finally this last fucntion takes the average of all the columns fomr the z score data frame and makes the headline index. 
        hf_df=self.combine_zscores().copy()
        hf_df=hf_df.ffill()
        hf_df[HF_Index_Name_string] = hf_df.mean(axis=1)
        hf_df = hf_df.reset_index()
        # Drop duplicates
        hf_df = hf_df.drop_duplicates(subset='index')
        # Set index back if necessary
        hf_df = hf_df.set_index('index')
        return hf_df
    

In [8]:
#Housing
USHBMIDX=Index_that_are_stationary('USHBMIDX INDEX')
NHSPATOT=Index_that_are_stationary('NHSPATOT INDEX')
SPCSUSS=Index_to_stationary('SPCSUSS INDEX')
CNSTRESI=Index_to_stationary('CNSTRESI INDEX')
MBAVBASC=Index_that_are_stationary('MBAVBASC INDEX')

housing_index=HF_Index()
housing_index.add_data(NHSPATOT)
housing_index.add_data(USHBMIDX)
housing_index.add_data(SPCSUSS)
housing_index.add_data(CNSTRESI)
housing_index.add_data(MBAVBASC)

In [9]:
#Business
NAPMALL=Index_that_are_stationary('NAPMALL INDEX')
NAPMNEWO=Index_that_are_stationary('NAPMNEWO INDEX')
NAPMNNO=Index_that_are_stationary('NAPMNNO INDEX')
RCHSINDX=Index_that_are_stationary('RCHSINDX INDEX')
RCSSSREV=Index_that_are_stationary('RCSSSREV INDEX')
DFEDGBA=Index_that_are_stationary('DFEDGBA INDEX')
DSERRVCC=Index_that_are_stationary('DSERRVCC INDEX')
EMPRGBCI=Index_that_are_stationary('EMPRGBCI INDEX')
CFNAI=Index_that_are_stationary('CFNAI INDEX')
MPMIUSCA=Index_that_are_stationary('MPMIUSCA INDEX')
OUMFGAF=Index_that_are_stationary('OUMFGAF INDEX')
KCLSSACI=Index_that_are_stationary('KCLSSACI INDEX')
business_sentiment=[NAPMALL, NAPMNEWO, NAPMNNO, RCHSINDX, RCSSSREV, DFEDGBA, DSERRVCC, EMPRGBCI, CFNAI, MPMIUSCA, OUMFGAF, KCLSSACI]
Businesss_index=HF_Index()
for i in business_sentiment:
    Businesss_index.add_data(i)

In [10]:
#Employment
intial_claims =Index_that_are_stationary('INJCJC INDEX')
intial_claims.invert_index_value() #inverts the dataframe values of the raw data
vacancies  =Index_to_stationary('JOLTPRIV INDEX')
HIREPRIR =Index_that_are_stationary('HIREPRIR INDEX')
QUITPRIR =Index_that_are_stationary('QUITPRIR INDEX')
layoffs =Index_that_are_stationary('CHALTOTL INDEX')
layoffs.invert_index_value()
household =Index_to_stationary('USLFTOT INDEX')
nfp_acyclical  =Index_to_stationary('USEETOTS INDEX')
USURTOT =Index_that_are_stationary('USURTOT INDEX')
USURTOT.invert_index_value()
AWH_TOTL =Index_that_are_stationary('AWH TOTL INDEX')
NAPMNEMP =Index_that_are_stationary('NAPMNEMP INDEX')
NAPMEMPL =Index_that_are_stationary('NAPMEMPL INDEX')
employment_list=[intial_claims, vacancies, HIREPRIR, QUITPRIR, layoffs, household, nfp_acyclical, USURTOT, AWH_TOTL, NAPMNEMP, NAPMEMPL]
Employment_index=HF_Index()
for i in employment_list:
    Employment_index.add_data(i)    

In [20]:
#Profits
spx_trail_eps=Datapoints_to_stationary("SPX INDEX", "TRAIL_12M_EPS")
spx_best_eps=Datapoints_to_stationary("SPX INDEX", "BEST_EPS")
spx_margin=Datapoints_to_stationary("SPX INDEX", "TRAIL_12M_PROF_MARGIN")
spx_sales=Datapoints_to_stationary("SPX INDEX", "BEST_NI_ADJ_TO_SALES")

mid_trail_eps=Datapoints_to_stationary("MID INDEX", "TRAIL_12M_EPS")
mid_best_eps=Datapoints_to_stationary("MID INDEX", "BEST_EPS")
mid_margin=Datapoints_to_stationary("MID INDEX", "TRAIL_12M_PROF_MARGIN")
mid_sales=Datapoints_to_stationary("MID INDEX", "BEST_NI_ADJ_TO_SALES")

sml_trail_eps=Datapoints_to_stationary("SML INDEX", "TRAIL_12M_EPS")
sml_best_eps=Datapoints_to_stationary("SML INDEX", "BEST_EPS")
sml_margin=Datapoints_to_stationary("SML INDEX", "TRAIL_12M_PROF_MARGIN")
sml_sales=Datapoints_to_stationary("SML INDEX", "BEST_NI_ADJ_TO_SALES")

bm7t_trail_eps=Datapoints_to_stationary("BM7T INDEX", "TRAIL_12M_EPS")
bm7t_best_eps=Datapoints_to_stationary("BM7T INDEX", "BEST_EPS")
bm7t_margin=Datapoints_to_stationary("BM7T INDEX", "TRAIL_12M_PROF_MARGIN")
bm7t_sales=Datapoints_to_stationary("BM7T INDEX", "BEST_NI_ADJ_TO_SALES")

b500xm7t_trail_eps=Datapoints_to_stationary("B500XM7T INDEX", "TRAIL_12M_EPS")
b500xm7t_best_eps=Datapoints_to_stationary("B500XM7T INDEX", "BEST_EPS")
b500xm7t_margin=Datapoints_to_stationary("B500XM7T INDEX", "TRAIL_12M_PROF_MARGIN")
b500xm7t_sales=Datapoints_to_stationary("B500XM7T INDEX", "BEST_NI_ADJ_TO_SALES")

GVADNCPA =Index_that_are_stationary('GVADNCPA INDEX')

profits_list = [spx_trail_eps,spx_best_eps,spx_margin,spx_sales,mid_trail_eps,mid_best_eps,mid_margin,mid_sales,sml_trail_eps,sml_margin,sml_sales,bm7t_trail_eps,bm7t_best_eps,bm7t_margin,bm7t_sales,b500xm7t_trail_eps,b500xm7t_best_eps,b500xm7t_margin,b500xm7t_sales]

Profits_index=HF_Index()
for i in profits_list:
    Profits_index.add_data(i)  
Profits_index.add_data(GVADNCPA)

In [22]:
#Credit
TDBCTOTL =Index_to_stationary('TDBCTOTL INDEX')
ALBNLOAN = Index_to_stationary('ALBNLOAN INDEX')
ALBNC_IL = Index_to_stationary('ALBNC&IL INDEX')
ALBNLLCN = Index_to_stationary('ALBNLLCN INDEX')
SLDETIGT = Index_to_stationary('SLDETIGT INDEX')
NFCINBON =Index_that_are_stationary('NFCINBON INDEX')
SBOIEXCR =Index_that_are_stationary('SBOIEXCR INDEX')
SBOILOAN =Index_that_are_stationary('SBOILOAN INDEX')

credit_list = [TDBCTOTL, ALBNLOAN,ALBNC_IL,ALBNLLCN ,SLDETIGT ,NFCINBON ,SBOIEXCR ,SBOILOAN] 

Credit_index=HF_Index()
for i in credit_list:
    Credit_index.add_data(i) 

In [23]:
#Consumer

CONSSENT =Index_that_are_stationary('CONSSENT INDEX')
USDECNLN =Index_that_are_stationary('USDECNLN INDEX')
CONCEXP =Index_that_are_stationary('CONCEXP INDEX')
MTSLRRT =Index_to_stationary('MTSLRRT$ INDEX')
CPI_YOY =Index_that_are_stationary('CPI YOY Index')
REDSMYOY =Index_that_are_stationary('REDSMYOY INDEX')

REDSMYOY_copy=REDSMYOY.index_value[REDSMYOY.ticker].copy() # here I need to rewrite the index data as my clas sisn't flexile enough to address changes like real retail sales
CPI_YOY_copy=CPI_YOY.index_value[CPI_YOY.ticker].copy()
REDSMYOY_copy=pd.merge(REDSMYOY_copy, CPI_YOY_copy, left_index=True, right_index=True)
REDSMYOY_copy['Real']=REDSMYOY_copy['PX_LAST_x']- REDSMYOY_copy['PX_LAST_y'] #here I am subtracting teh change in y/y retail sales by CPI y/y to get real retail sales y/y
REDSMYOY.index_value[REDSMYOY.ticker] = REDSMYOY_copy['Real']

PCE_CONC =Index_to_stationary('PCE CONC INDEX')

PCE_CONC_trend =Index_that_are_stationary('PCE CONC INDEX') #this process is me getting the trend in PCE consumption by a simple regression on time and them editing the raw data to be process into sationary
model = LinearRegression()
x = np.arange(len(PCE_CONC_trend.index_value.dropna().index)).reshape(-1, 1)
y = PCE_CONC_trend.index_value.dropna().values
model.fit(x, y)
# Extract the trend
PCE_CONC_trend_copy=PCE_CONC_trend.index_value[PCE_CONC_trend.ticker].copy().dropna()
PCE_CONC_trend_copy["Trend"] = model.predict(x)
PCE_CONC_trend_copy['Deviation from Trend'] = PCE_CONC_trend_copy["PX_LAST"] - PCE_CONC_trend_copy["Trend"]
PCE_CONC_trend.index_value[PCE_CONC_trend.ticker] = PCE_CONC_trend_copy['Deviation from Trend']

consumer_list = [CONSSENT,CONCEXP,USDECNLN,MTSLRRT,REDSMYOY ,PCE_CONC ,PCE_CONC_trend] 

Consumer_index=HF_Index()
for i in consumer_list:
    Consumer_index.add_data(i) 



In [24]:
#Fiscal
gov_expend =Index_to_stationary('ECOGBUSN INDEX') #fiscal index is % of GDP so I had to alter the raw data as well for teh classes to work. 
gdp =Index_to_stationary('GDP CUR$ INDEX')
fiscal = Index_that_are_stationary('ECOGBUSN INDEX')
gov_expend_copy=gov_expend.index_value[gov_expend.ticker].copy()
gdp_copy=gdp.index_value[gdp.ticker].copy()
fiscal_deficit = gov_expend_copy/gdp_copy
gov_expend.index_value[gov_expend.ticker] = fiscal_deficit
fiscal.index_value[fiscal.ticker]=gov_expend.mean_revert_trans()["PX_LAST"]-gov_expend.mean_revert_trans()["6_months_ago"]
fiscal.invert_index_value()

Fiscal_index=HF_Index()
Fiscal_index.add_data(fiscal)

In [25]:
#Geo Politics
geo_politics = Haver.data('DGLGPRD', database='ESG', startdate='1970-01-01',frequency = 'caldaily') #Geopoltics is ahver data, but I wrote the class for Bloomberg, so I'm just changing the data to match bloomberg
geo_politics=geo_politics.to_timestamp().reset_index()
days_between = (pd.to_datetime('today').normalize() - geo_politics["index"].iloc[-1]).days
day_between_market = int(days_between-(days_between/7*2))
geo_politics["index"] = geo_politics["index"] + timedelta(days=(pd.to_datetime('today').normalize() - geo_politics["index"].iloc[-1]).days)
geo_politics=geo_politics.set_index("index")
geo_politics=geo_politics.shift(days_between*-1)
Geo_Politics =Index_that_are_stationary('APEZCOGR Index')
Geo_Politics_copy=Geo_Politics.index_value[Geo_Politics.ticker].copy()
Geo_Politics_copy=pd.merge(Geo_Politics_copy, geo_politics, left_index=True, right_index=True)
Geo_Politics.index_value[Geo_Politics.ticker] = Geo_Politics_copy['dglgprd']

GeoPolitical_index=HF_Index()
GeoPolitical_index.add_data(Geo_Politics)

In [26]:
#Delinquencies/Bankruptcies
USEQTATO =Index_that_are_stationary('USEQTATO INDEX')
USEQTLTO =Index_that_are_stationary('USEQTLTO INDEX')
USEQCCTO =Index_that_are_stationary('USEQCCTO INDEX')
BNKRINDX =Index_that_are_stationary('BNKRINDX INDEX')

delinquencies_list=[USEQTATO, USEQTLTO, USEQCCTO, BNKRINDX]
Delinquencies_Bankruptcies_index=HF_Index()
for i in delinquencies_list:
    Delinquencies_Bankruptcies_index.add_data(i)

In [27]:
#Inflation
NAPMPRIC =Index_that_are_stationary('NAPMPRIC INDEX')
NAPMNPRC =Index_that_are_stationary('NAPMNPRC INDEX')
FRGHGLOB =Index_to_stationary('FRGHGLOB INDEX')
PCE_DEF =Index_to_stationary('PCE DEF INDEX')
FDIUFDA =Index_to_stationary('FDIUFDA INDEX')
FDIUSGA =Index_to_stationary('FDIUSGA INDEX')
AHE_TOTL =Index_to_stationary('AHE TOTL INDEX')
AWP_TOTL =Index_to_stationary('AWP TOTL INDEX')
ECICCIVL =Index_to_stationary('ECICCIVL INDEX')
BCOMEN =Index_to_stationary('BCOMEN INDEX')
BCOMIN =Index_to_stationary('BCOMIN INDEX')
BCOMAG =Index_to_stationary('BCOMAG INDEX')
COSTNFRM =Index_to_stationary('COSTNFRM INDEX')

inflation_list=[NAPMPRIC, NAPMNPRC, FRGHGLOB, PCE_DEF, NAPMPRIC,NAPMNPRC, FRGHGLOB, PCE_DEF, FDIUFDA, FDIUSGA, AHE_TOTL, AWP_TOTL, ECICCIVL, BCOMEN, BCOMIN, BCOMAG, COSTNFRM ]
Inflation_index=HF_Index()
for i in inflation_list:
    Inflation_index.add_data(i)

In [30]:
#Financial conditions
GSUSFCI =Index_that_are_stationary('GSUSFCI INDEX')
BICLB10Y =Index_that_are_stationary('BICLB10Y INDEX')
long_yields =Index_that_are_stationary('USGG10YR INDEX')
CPI_INDX =Index_to_stationary('CPI INDX INDEX')
fed =Index_that_are_stationary('FDTR INDEX')
BFCIUS =Index_that_are_stationary('BFCIUS INDEX')
BFCIUS.invert_index_value()
NFCIINDX =Index_that_are_stationary('NFCIINDX INDEX')
KCFSINDX =Index_that_are_stationary('KCFSINDX INDEX')
FRNYCMDI =Index_that_are_stationary('FRNYCMDI INDEX')
FCIGMB1Y =Index_that_are_stationary('FCIGMB1Y INDEX')
USNRUS =Index_that_are_stationary('USNRUS INDEX')
FCIGMB3Y =Index_that_are_stationary('FCIGMB3Y INDEX')

long_yields_copy=long_yields.index_value[long_yields.ticker].copy() #here i'm getting the changes in long yields by taking away the natural rate of interest and inflation.  So growth expectations
CPI_INDX_copy=CPI_INDX.mean_revert_trans()[['6_month_change']].copy()*100
USNRUS_copy=USNRUS.index_value[USNRUS.ticker].copy()
long_yields_copy=pd.merge(long_yields_copy, USNRUS_copy, left_index=True, right_index=True)
long_yields_copy=pd.merge(long_yields_copy, CPI_INDX_copy, left_index=True, right_index=True)
long_yields_copy['long_yields']=long_yields_copy['PX_LAST_x']- long_yields_copy['PX_LAST_y']-long_yields_copy['6_month_change']
long_yields.index_value[long_yields.ticker] = long_yields_copy['long_yields']

fed_copy = fed.index_value[fed.ticker].copy() #here I'm altering the rw data to relfect the restcitivness of real policy rate
fed_copy=pd.merge(fed_copy, USNRUS_copy, left_index=True, right_index=True)
fed_copy=pd.merge(fed_copy, CPI_INDX_copy, left_index=True, right_index=True)
fed_copy['long_yields']=fed_copy['PX_LAST_x']- fed_copy['PX_LAST_y']-fed_copy['6_month_change']
fed.index_value[fed.ticker] = fed_copy['long_yields']

fc_list=[GSUSFCI, 	BICLB10Y, 	long_yields, fed, 	BFCIUS, 	NFCIINDX, 	KCFSINDX, 	FRNYCMDI, 	FCIGMB3Y ,FCIGMB1Y ]
Financial_conditions_index=HF_Index()
for i in fc_list:
    Financial_conditions_index.add_data(i)

In [31]:
#Market Bubble Risk

SPX_PE=Datapoints_that_are_stationary("SPX Index", "PE_RATIO")
CHG_SPX_PE=Datapoints_to_stationary("SPX Index", "PE_RATIO")
SPX_eps=Datapoints_to_stationary("SPX INDEX", "TRAIL_12M_EPS")
real_spx_eps=Datapoints_to_stationary("SPX INDEX", "TRAIL_12M_EPS")
fundamental=Datapoints_that_are_stationary("SPX INDEX", "TRAIL_12M_EPS")
CPI_INDX =Index_to_stationary('CPI INDX INDEX')
SPX =Index_to_stationary('SPX INDEX')
concentration =Index_to_stationary('SPX INDEX')
real_spx =Index_to_stationary('SPX INDEX')
SPW =Index_to_stationary('SPW INDEX')
AAA =Index_that_are_stationary('BASPCAAA INDEX')
BBB =Index_that_are_stationary('CSI BBB INDEX')
CCC =Index_that_are_stationary('CSI BARC INDEX')

real_price = real_spx.index_value[real_spx.ticker].copy() #here i'm replacing the SPX index with an inflation adjusted one
CPI_INDX_copy=CPI_INDX.index_value[CPI_INDX.ticker].copy()
real_price = pd.merge(real_price, CPI_INDX_copy, left_index=True, right_index=True)
real_price['Real SPX']=real_price['PX_LAST_x']/real_price['PX_LAST_y']
real_spx.index_value[real_spx.ticker] = real_price['Real SPX']

real_eps = real_spx_eps.index_value[real_spx_eps.ticker].copy() #here I'm replacing eps with real eps
real_eps = pd.merge(real_eps, CPI_INDX_copy, left_index=True, right_index=True)
real_eps['Real EPS']=real_eps['TRAIL_12M_EPS']/real_eps['PX_LAST']
real_spx_eps.index_value[real_spx.ticker] = real_eps['Real EPS']*100

SPX_eps_copy=SPX_eps.mean_revert_trans()[['6_month_change']].copy()*100 # I'm making a varaible called fudnamnetal, which 
SPX_eps_copy_abs = SPX_eps_copy.copy().abs()
CHG_SPX_PE_copy=CHG_SPX_PE.mean_revert_trans()[['6_month_change']].copy()*100
CHG_SPX_PE_copy = CHG_SPX_PE_copy.abs()
SPX_eps_copy = pd.merge(SPX_eps_copy, SPX_eps_copy, left_index=True, right_index=True)
SPX_eps_copy = pd.merge(SPX_eps_copy, CHG_SPX_PE_copy, left_index=True, right_index=True)
SPX_eps_copy['Fundamental'] = (SPX_eps_copy['6_month_change_x']/(SPX_eps_copy['6_month_change_y']+SPX_eps_copy['6_month_change']))*-1
fundamental.index_value[fundamental.ticker] = SPX_eps_copy['Fundamental']

concentration_copy = concentration.mean_revert_trans()[['6_month_change']].copy()*100 #this measures market concentration 
SPW_copy=SPW.mean_revert_trans()[['6_month_change']].copy()*100
concentration_copy = pd.merge(concentration_copy, SPW_copy, left_index=True, right_index=True)
concentration_copy['Concentration']=concentration_copy['6_month_change_x']-concentration_copy['6_month_change_y']
concentration.index_value[concentration.ticker] = concentration_copy['Concentration']

bubble_list=[SPX_PE, 	fundamental, 	concentration, AAA, 	BBB, 	CCC]
Bubble_index=HF_Index()
for i in bubble_list:
    Bubble_index.add_data(i)

In [32]:
def convert_to_daily(df): #this is code aimed at converting a mnthly frequency to daily, needed for haver data
    df_daily = df.copy()
    df_daily = df_daily.to_timestamp()
    df_daily.index = df_daily.index.to_period('M').to_timestamp('D')
    #df_daily = df_daily.resample('D').ffill()
    return df_daily

In [34]:
SPECAPE = Haver.data('SPECAPE', database='USECON', startdate='1970-01-01') #this is all work to get eh Cyclcially adjusted PE ratio from haver to daily
CAPE=Datapoints_that_are_stationary("SPX Index", "PE_RATIO")
specape_daily=convert_to_daily(SPECAPE).reset_index()
specape_daily["index"] = specape_daily["index"] + timedelta(days=(pd.to_datetime('today').normalize() - specape_daily["index"].iloc[-1]).days)
specape_daily=specape_daily.set_index("index")
specape_daily = specape_daily.resample('D').ffill()
CAPE.index_value[CAPE.ticker] = specape_daily['specape']
Bubble_index.add_data(CAPE)

In [37]:
#finally we bring all the indicators together to create the table of headline indicies 
housing_index_indicator=housing_index.highfrequency_index("Housing Index")
Businesss_index_indicator=Businesss_index.highfrequency_index("Business Index")
Employment_index_indicator=Employment_index.highfrequency_index("Employment Index")
Profits_index_indicator=Profits_index.highfrequency_index("Profits Index")
Credit_index_indicator=Credit_index.highfrequency_index("Credit Index")
Consumer_index_indicator=Consumer_index.highfrequency_index("Consumer Index")
Fiscal_index_indicator=Fiscal_index.highfrequency_index("Fiscal Index")
GeoPolitical_index_indicator=GeoPolitical_index.highfrequency_index("Geo Politics Index")
Delinquencies_Bankruptcies_index_indicator=Delinquencies_Bankruptcies_index.highfrequency_index("Solvency Index")
Inflation_index_indicator=Inflation_index.highfrequency_index("Inflation Index")
Financial_conditions_index_indicator=Financial_conditions_index.highfrequency_index("Financial Conditions Index")
Bubble_index_indicator=Bubble_index.highfrequency_index("Bubble Index")
bubble_cols = ["S&P PE", 	"Fundamental", 	"Concentration", "AAA Spreads", 	"BBB Spread", 	"CCC Spreads","CAPE", "Bubble Index"]
Bubble_index_indicator.columns = bubble_cols

HF_indicators_df = housing_index_indicator.iloc[:,-1:] #the last column of each indictor dataframe is the headline indicator. 

index_list = [Businesss_index_indicator,Employment_index_indicator,Profits_index_indicator,Credit_index_indicator,Consumer_index_indicator,Fiscal_index_indicator,
              GeoPolitical_index_indicator,Delinquencies_Bankruptcies_index_indicator,Inflation_index_indicator,Financial_conditions_index_indicator,Bubble_index_indicator]
for i in index_list:
    HF_indicators_df = pd.merge(HF_indicators_df, i.iloc[:,-1:], left_index=True, right_index=True)

C:\Users\limacinnis\AppData\Local\anaconda3\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
